In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import numpy as np
import pandas as pd
import sklearn
import nltk
nltk.download('stopwords')
nltk.download('punkt')
from sklearn import feature_extraction,model_selection
from sklearn.metrics import confusion_matrix
import glob
import os
import string
import tensorflow as tf
import keras as K
import zipfile
from itertools import chain
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import matplotlib.pyplot as plt
from sklearn.metrics import plot_confusion_matrix, classification_report

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [0]:
# importing Word2Vec
from gensim.models import Word2Vec
from gensim.models import FastText

In [0]:
X=pd.read_csv('/content/drive/My Drive/Hindi_Combined.csv')
X.head()

Unnamed: 0                                               Text  Label
0           0  RANDTV tumhare najayaz baap is area hai ki wah...      2
1           1        First stage par dus jootey khaye Grover  se      2
2           2  Salman aur aamir ki kounsi movie release huyee...      2
3           3  Wk aur bhakt aa gya.  Abe americans ka bolna h...      1
4           4  Beta to tu apne baap ka hai ...permission to t...      0

In [0]:
df1=pd.read_csv('/content/drive/My Drive/Colab Notebooks/hin_dev.csv',index_col='ID')
df1.head()

Text  ... Sub-task B
ID                                                          ...           
C38.9    bkl interviewers kuch jaada hi open minded bnt...  ...       NGEN
C4.1510  Bhaiya shaadi mein zaroor aana movie ka plot j...  ...       NGEN
C19.95   Section 375 hai kya??? .... Ye to batate kam s...  ...       NGEN
C4.281   कबीर सिंह hit Hui इससे पता चलता है आजकल के लोग...  ...       NGEN
C4.216              Maine itni kam dislike kbhi nhii dekhi  ...       NGEN

[5 rows x 3 columns]

In [0]:
Y=df1.drop(columns={'Sub-task B'})
type(Y['Sub-task A'])

pandas.core.series.Series

In [0]:
X.head()

Unnamed: 0                                               Text  Label
0           0  RANDTV tumhare najayaz baap is area hai ki wah...      2
1           1        First stage par dus jootey khaye Grover  se      2
2           2  Salman aur aamir ki kounsi movie release huyee...      2
3           3  Wk aur bhakt aa gya.  Abe americans ka bolna h...      1
4           4  Beta to tu apne baap ka hai ...permission to t...      0

In [0]:
Y['Sub-task A']=Y['Sub-task A'].map({'OAG':2,'CAG':1,'NAG':0})
Y.head()

Text  Sub-task A
ID                                                                    
C38.9    bkl interviewers kuch jaada hi open minded bnt...           2
C4.1510  Bhaiya shaadi mein zaroor aana movie ka plot j...           0
C19.95   Section 375 hai kya??? .... Ye to batate kam s...           0
C4.281   कबीर सिंह hit Hui इससे पता चलता है आजकल के लोग...           2
C4.216              Maine itni kam dislike kbhi nhii dekhi           0

In [0]:
X_train=X['Text']
X_train

0       RANDTV tumhare najayaz baap is area hai ki wah...
1             First stage par dus jootey khaye Grover  se
2       Salman aur aamir ki kounsi movie release huyee...
3       Wk aur bhakt aa gya.  Abe americans ka bolna h...
4       Beta to tu apne baap ka hai ...permission to t...
                              ...                        
9144    Asexual h.. bisexual... homosexual... bhai ase...
9145    Video pura dekne ke pahile hi mai bhai ke vide...
9146                     konsa place hai bhai ...nam bolo
9147                             Kuch zada hi likh diya 🙄
9148    scary tube par kya woh karna sahi tha? Pucha k...
Name: Text, Length: 9149, dtype: object

In [0]:
y_train=X['Label']
y_train

0       2
1       2
2       2
3       1
4       0
       ..
9144    0
9145    0
9146    0
9147    0
9148    0
Name: Label, Length: 9149, dtype: int64

In [0]:
X_test=Y['Text']
X_test

ID
C38.9        bkl interviewers kuch jaada hi open minded bnt...
C4.1510      Bhaiya shaadi mein zaroor aana movie ka plot j...
C19.95       Section 375 hai kya??? .... Ye to batate kam s...
C4.281       कबीर सिंह hit Hui इससे पता चलता है आजकल के लोग...
C4.216                  Maine itni kam dislike kbhi nhii dekhi
                                   ...                        
C29.18                   Are bhai konsi duniya mai rehate ho😂😂
C9.225                                            Sahi to bola
C7.1685                                Or tum Kay Kar raya ho?
C36.692.1    Ye bhaat sahi hain par zyada dhin thine ga nah...
C4.1202                   Islye maine Kabaad singh nai dekha..
Name: Text, Length: 997, dtype: object

In [0]:
y_test=Y['Sub-task A']
y_test

ID
C38.9        2
C4.1510      0
C19.95       0
C4.281       2
C4.216       0
            ..
C29.18       0
C9.225       0
C7.1685      0
C36.692.1    0
C4.1202      0
Name: Sub-task A, Length: 997, dtype: int64

In [0]:
X_train.shape,X_test.shape,y_train.shape,y_test.shape

((9149,), (997,), (9149,), (997,))

In [0]:
def create_tokens(lines):
  #stop_words = set(nltk.corpus.stopwords.words('english'))
  t_line=lines.apply(lambda x: nltk.word_tokenize(x))
  #t_line=t_line.apply(lambda x: [w for w in x if not w in stop_words])
  return t_line

In [0]:
X_train=create_tokens(X_train)
print(X_train)
tokenizer= Tokenizer()
tokenizer.fit_on_texts(X_train)
tokenizer


0       [RANDTV, tumhare, najayaz, baap, is, area, hai...
1       [First, stage, par, dus, jootey, khaye, Grover...
2       [Salman, aur, aamir, ki, kounsi, movie, releas...
3       [Wk, aur, bhakt, aa, gya, ., Abe, americans, k...
4       [Beta, to, tu, apne, baap, ka, hai, ..., permi...
                              ...                        
9144    [Asexual, h.., bisexual, ..., homosexual, ...,...
9145    [Video, pura, dekne, ke, pahile, hi, mai, bhai...
9146            [konsa, place, hai, bhai, ..., nam, bolo]
9147                      [Kuch, zada, hi, likh, diya, 🙄]
9148    [scary, tube, par, kya, woh, karna, sahi, tha,...
Name: Text, Length: 9149, dtype: object


In [0]:
length=np.average(X_train.apply(lambda x:len(x)*len(x)))
length=np.sqrt(length)
length=48

In [0]:
def encode_text(tokenizer, lines, length):
  encoded = tokenizer.texts_to_sequences(lines)
  padded = pad_sequences(encoded, maxlen=length, padding='post')
  return padded

In [0]:
word_index=tokenizer.word_index

In [0]:
X_test=create_tokens(X_test)
print(X_test)

ID
C38.9        [bkl, interviewers, kuch, jaada, hi, open, min...
C4.1510      [Bhaiya, shaadi, mein, zaroor, aana, movie, ka...
C19.95       [Section, 375, hai, kya, ?, ?, ?, ..., ., Ye, ...
C4.281       [कबीर, सिंह, hit, Hui, इससे, पता, चलता, है, आज...
C4.216          [Maine, itni, kam, dislike, kbhi, nhii, dekhi]
                                   ...                        
C29.18           [Are, bhai, konsi, duniya, mai, rehate, ho😂😂]
C9.225                                        [Sahi, to, bola]
C7.1685                       [Or, tum, Kay, Kar, raya, ho, ?]
C36.692.1    [Ye, bhaat, sahi, hain, par, zyada, dhin, thin...
C4.1202            [Islye, maine, Kabaad, singh, nai, dekha..]
Name: Text, Length: 997, dtype: object


In [0]:
X_train=encode_text(tokenizer,X_train,length)
X_train
#pd.DataFrame(train_x).to_csv("train_file.csv")

array([[12745,   589,  8238, ...,     0,     0,     0],
       [ 2590,  6237,    83, ...,     0,     0,     0],
       [ 1480,    28,  6238, ...,     0,     0,     0],
       ...,
       [ 3946,  5475,     2, ...,     0,     0,     0],
       [   59,  4754,    29, ...,     0,     0,     0],
       [12471,  6207,    83, ...,     0,     0,     0]], dtype=int32)

In [0]:
X_test=encode_text(tokenizer,X_test,length)
X_test
#pd.DataFrame(train_x).to_csv("train_file.csv")

array([[  59, 7996,   29, ...,    0,    0,    0],
       [ 843, 2618,   79, ...,    0,    0,    0],
       [1637, 1345,    2, ...,    0,    0,    0],
       ...,
       [  47,   85,  926, ...,    0,    0,    0],
       [  26, 6231,   82, ...,    0,    0,    0],
       [8248,  408,  125, ...,    0,    0,    0]], dtype=int32)

In [0]:
from keras.utils.np_utils import to_categorical
y_test = to_categorical(np.asarray(y_test))
y_train = to_categorical(np.asarray(y_train))
print(y_test)
print(y_train)

[[0. 0. 1.]
 [1. 0. 0.]
 [1. 0. 0.]
 ...
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]]
[[0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]
 ...
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]]


In [0]:
# print(len(X_train))
# print(len(X_test))
# print(len(y_train))
# print(len(y_test))
X_train.shape,X_test.shape,y_train.shape,y_test.shape

((9149, 48), (997, 48), (9149, 3), (997, 3))

In [0]:
# Creating embedding matrix
# Using 
import zipfile
zip_ref = zipfile.ZipFile("/content/drive/My Drive/crawl-300d-2M.vec.zip", 'r')
zip_ref.extractall()
zip_ref.close()

In [0]:
#load pre-trained embedding:
embeddings_index = {}
f = open('/content/crawl-300d-2M.vec', encoding="utf8")
for line in f:
    values = line.split()#This splits each values in a  line with spaces
    word = values[0] # This store the first values which is the word in english
    coefs = np.asarray(values[1:], dtype='float32') # This converts all splitted values from 1 to last(expect first word) into an array
    embeddings_index[word] = coefs # english word = array of numerical values
f.close()
print('Total %s word vectors in crawl-300-2M.' % len(embeddings_index))

Total 1999996 word vectors in crawl-300-2M.


In [0]:
print((X_train))
print((X_test))
print((y_train))
print((y_test))

[[12745   589  8238 ...     0     0     0]
 [ 2590  6237    83 ...     0     0     0]
 [ 1480    28  6238 ...     0     0     0]
 ...
 [ 3946  5475     2 ...     0     0     0]
 [   59  4754    29 ...     0     0     0]
 [12471  6207    83 ...     0     0     0]]
[[  59 7996   29 ...    0    0    0]
 [ 843 2618   79 ...    0    0    0]
 [1637 1345    2 ...    0    0    0]
 ...
 [  47   85  926 ...    0    0    0]
 [  26 6231   82 ...    0    0    0]
 [8248  408  125 ...    0    0    0]]
[[0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]
 ...
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]]
[[0. 0. 1.]
 [1. 0. 0.]
 [1. 0. 0.]
 ...
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]]


In [0]:
# Initialize the weight for each word for the embedding layer. Words not present
# in the dictionary initialized by random value 
EMBEDDING_DIM = 300
embedding_matrix = np.random.random((len(word_index) + 1, EMBEDDING_DIM))
#embedding_matrix = (16433, 100) NOTE: It is from uniques words in our tweets
for word, i in word_index.items(): # Selecting word from our tweet unique words list for changing with the values from glove
    embedding_vector = embeddings_index.get(word) # Finding our tweet words in glove and taking its array of 100 values
    if embedding_vector is not None: 
#words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector
embedding_matrix.shape

(33453, 300)

In [0]:
from keras.utils.vis_utils import plot_model
from keras.models import Sequential
from keras.layers import Input,LSTM
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers import Embedding
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
model=Sequential()
model.add(Embedding(input_dim = 33453, output_dim = 300, weights=[embedding_matrix], input_length=length,trainable=False))
model.add(LSTM(192, dropout=0.3, recurrent_dropout=0.2))
model.add(Dense(3,activation='softmax'))
model.compile(loss = 'binary_crossentropy', optimizer='adam',metrics = ['accuracy'])
model.summary()

Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_8 (Embedding)      (None, 48, 300)           10035900  
_________________________________________________________________
lstm_1 (LSTM)                (None, 192)               378624    
_________________________________________________________________
dense_16 (Dense)             (None, 3)                 579       
Total params: 10,415,103
Trainable params: 379,203
Non-trainable params: 10,035,900
_________________________________________________________________


In [0]:
model.fit(X_train, y_train, validation_data = (X_test, y_test),epochs=100,batch_size=256, verbose=2)

Train on 9149 samples, validate on 997 samples
Epoch 1/100
 - 4s - loss: 0.6353 - acc: 0.6667 - val_loss: 0.6213 - val_acc: 0.6650
Epoch 2/100
 - 4s - loss: 0.6335 - acc: 0.6665 - val_loss: 0.6138 - val_acc: 0.6653
Epoch 3/100
 - 4s - loss: 0.6334 - acc: 0.6670 - val_loss: 0.6111 - val_acc: 0.6663
Epoch 4/100
 - 4s - loss: 0.6221 - acc: 0.6717 - val_loss: 0.5570 - val_acc: 0.7449
Epoch 5/100
 - 4s - loss: 0.6042 - acc: 0.6867 - val_loss: 0.5314 - val_acc: 0.7636
Epoch 6/100
 - 4s - loss: 0.5842 - acc: 0.6979 - val_loss: 0.5364 - val_acc: 0.7566
Epoch 7/100
 - 4s - loss: 0.5663 - acc: 0.7063 - val_loss: 0.5078 - val_acc: 0.7589
Epoch 8/100
 - 4s - loss: 0.5612 - acc: 0.7088 - val_loss: 0.4869 - val_acc: 0.7660
Epoch 9/100
 - 4s - loss: 0.5501 - acc: 0.7131 - val_loss: 0.4714 - val_acc: 0.7723
Epoch 10/100
 - 4s - loss: 0.5447 - acc: 0.7167 - val_loss: 0.4734 - val_acc: 0.7730
Epoch 11/100
 - 4s - loss: 0.5371 - acc: 0.7234 - val_loss: 0.4715 - val_acc: 0.7793
Epoch 12/100
 - 4s - loss: 

In [0]:
predictions_test = model.predict(X_test)

predictions_test

array([[4.4999853e-02, 9.5128965e-01, 3.7105214e-03],
       [9.9912983e-01, 8.6782844e-04, 2.3861662e-06],
       [1.3095475e-02, 6.9301033e-01, 2.9389426e-01],
       ...,
       [9.6557647e-01, 1.8697701e-02, 1.5725823e-02],
       [4.2555422e-02, 9.0293944e-01, 5.4505102e-02],
       [7.4298805e-01, 6.5936898e-03, 2.5041825e-01]], dtype=float32)

In [0]:
pred_test=np.zeros_like(predictions_test)
pred_test[np.arange(len(predictions_test)),predictions_test.argmax(1)]=1
output_DataFrame=pd.DataFrame(pred_test.argmax(axis=1),index=df1.index)
output_DataFrame[0]=output_DataFrame[0].map({0:'NAG',1:'CAG',2:'OAG'})
output_DataFrame.to_csv("Hindi Subtask B ou.csv")

In [0]:
from sklearn.metrics import classification_report
print(classification_report(y_test,pred_test))

              precision    recall  f1-score   support

           0       0.79      0.72      0.76       578
           1       0.36      0.46      0.40       211
           2       0.52      0.49      0.50       208

   micro avg       0.62      0.62      0.62       997
   macro avg       0.56      0.56      0.55       997
weighted avg       0.64      0.62      0.63       997
 samples avg       0.62      0.62      0.62       997



In [0]:
df_test=pd.read_csv("/content/drive/My Drive/Colab Notebooks/hin_test.csv")
df_test.head()

ID                                        Text
0  C52.17                                          ko
1  C52.39                              ladkiyon video
2  C52.73                             ki video gahrep
3   C60.3                       o sadharon video bhai
4  C60.43  ba bhai kyea bola tum moza aaa giea 😌😌😌😂😂😂

In [0]:
model.save('HindiTaskB.h5')

In [0]:
test_text=encode_text(tokenizer,df_test['Text'],length)
test_text

array([[   7,    0,    0, ...,    0,    0,    0],
       [3229,  126,    0, ...,    0,    0,    0],
       [   6,  126,    0, ...,    0,    0,    0],
       ...,
       [ 271, 1039, 8299, ...,  159,  869, 4289],
       [1574, 2566,   34, ..., 4018, 4567,  311],
       [3443,  554,  168, ...,    0,    0,    0]], dtype=int32)

In [0]:
predictions_test = model.predict(test_text)

predictions_test

array([[8.7135029e-01, 2.9158773e-02, 9.9490970e-02],
       [9.8230928e-01, 2.9442701e-03, 1.4746461e-02],
       [9.7268361e-01, 1.0196120e-02, 1.7120333e-02],
       ...,
       [7.1363673e-02, 5.2094120e-03, 9.2342693e-01],
       [7.7121731e-06, 5.6996714e-07, 9.9999177e-01],
       [9.5991891e-07, 5.2664673e-05, 9.9994636e-01]], dtype=float32)

In [0]:
pred_test=np.zeros_like(predictions_test)
pred_test[np.arange(len(predictions_test)),predictions_test.argmax(1)]=1
output_DataFrame=pd.DataFrame(pred_test.argmax(axis=1),index=df_test['ID'])
output_DataFrame[0]=output_DataFrame[0].map({0:'NAG',1:'CAG',2:'OAG'})
output_DataFrame.to_csv("Hindi Subtask B CNN Fasttext output.csv")